![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/taller_3/main/figs/taller-meca-aprendizaje%20no%20supervisado_banner%201169%20x%20200%20px%20-05.png)

# Caso-taller:  Recomendando el Blog de  Hernán Casciari


[Hernán Casciari](https://hernancasciari.com/#bio), es un escritor argentino, que escribe blog posts con cuentos e historias  relacionadas con el futbol, su vida, infancia, y relaciones familiares con toques de ficción. Este [blog](https://hernancasciari.com/blog/) es  tan interesantes que en 2005 fue premiado como “El mejor blog del mundo” por Deutsche Welle de Alemania.

El objetivo de este caso-taller es construir un sistema de recomendación basado en los contenidos de los posts utilizando similitud de las palabras usadas o temas de los cuentos.

## Instrucciones generales

1. Para desarrollar el *cuaderno*, primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos

En la carpeta `data` se encuentran el archivo `blog_casciari.csv` con el título, la fecha de publicación, y el contenido de los cuentos publicados en el blog  de sr. Casciari. Cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.
   

In [ ]:
!pip install --upgrade --force-reinstall spicy numpy gensim pandas

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17

In [4]:
pip install -q pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.7 MB/s eta 0:00:00


In [5]:
!pip install -U pip setuptools wheel
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [6]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 82.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
# cargamos librerias
import pandas as pd
import numpy as np
import os

import re
from spellchecker import SpellChecker
import unicodedata

import spacy
from spacy.language import Language

import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
url = 'https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/blog_casciari.csv'

In [9]:
blog = pd.read_csv(url)
blog.head(10)

,titulo,fecha,cuento
0,El rincón blanco,1/11/08,De pronto yo estaba en el hogar donde pasé la ...
1,Mínimos avances en la cama,1/24/08,"Menos la cama, todo ha mejorado en este mundo...."
2,Don Marcos,2/19/08,"Dos veces, y no una, mi abuelo materno me ayud..."
3,Los dos rulfos,3/26/08,"A su regreso de México, mi amigo Comequechu no..."
4,La noticia no es el perro,4/15/08,"De repente, un video de You Tube recibe un mil..."
5,El milagro de los pueblos,4/22/08,Cuando nació la Nina no tuve ganas de escribir...
6,Ropa sucia,5/6/08,Ya de entrada caí mal parado. Vine al mundo ju...
7,La Última Gran Guerra del Hombre Chiquito,5/13/08,Años atrás mi tataranieto Woung comenzó a envi...
8,Instrucciones para crear mundos paralelos,5/20/08,"Descubrimos el truco por casualidad, en nuestr..."
9,Una década flaca,5/27/08,Una tarde de 1990 fui a tomar la leche a la ca...


### 2. Homogenización de textos

Para cumplir con el objetivo de generar recomendaciones en esta sección debe preparar los posts para poder ser utilizados en su sistema de recomendación. Para ello, "limpie" y "tokenize" cada uno de los cuentos, describiendo detalladamente los pasos que realizo y si transformó o eliminó ciertas palabras. Para asistirlo en la tarea he creado listas de *stopwords* que están disponibles en la carpeta `data`. En su procedimiento ilustre la limpieza con el cuento 'La venganza del metegol'. (En su limpieza recuerde que el objetivo es generar recomendaciones a partir de la similitud de las palabras o temas de los cuentos)

No hemos podido limpiar todo el DataFrame al tiempo. Proponemos dividirlo en 5 bloques de 104 observaciones cada uno y, así, limpiarlo

In [16]:
num_bloques = 5
tamano_bloque = 104

lista_dfs = [blog[i * tamano_bloque:(i + 1) * tamano_bloque] for i in range(num_bloques)]

In [ ]:
# ruta actual
base_dir = os.getcwd()

# ruta a la carpeta "data"
data_dir = os.path.join(base_dir, "data")

# ruta final con subcarpeta "datos"
output_dir = os.path.join(data_dir, "datos")

# crear la carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

for i, df in enumerate(lista_dfs):
  nombre_archivo = f'datos/blog_bloque_{i + 1}.csv'
  df.to_csv(os.path.join(data_dir, nombre_archivo), index=False)

In [ ]:
blog.info()
# no tenemos problemas con missing alues

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  520 non-null    object
 1   fecha   520 non-null    object
 2   cuento  520 non-null    object
dtypes: object(3)
memory usage: 12.3+ KB


In [17]:
spell = SpellChecker(language='es')

import requests

# Añadir palabras correctas al diccionario para evitar falsos positivos
url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/es/es_50k.txt"
response = requests.get(url)
with open("vocab.txt", "wb") as f:
    f.write(response.content)


with open('vocab.txt', 'r', encoding='utf-8') as f:
    palabras_externas = [line.strip().split()[0] for line in f]

spell.word_frequency.load_words(palabras_externas)

def corregir_texto(texto):
    # Separar palabras y signos de puntuación
    tokens = re.findall(r'\w+|[^\w\s]', texto, re.UNICODE)

    corregidas = []
    for token in tokens:
        if token.isalpha():
            if token not in spell:
                correccion = spell.correction(token)
                corregidas.append(correccion if correccion is not None else token)
            else:
                corregidas.append(token)
        else:
            corregidas.append(token)  # Mantener puntuación tal cual

    # Reconstruir el texto respetando espacios entre palabras y no entre signos
    texto_corregido = ''
    for i, token in enumerate(corregidas):
        if i > 0 and token not in ',.!?':
            texto_corregido += ' '
        texto_corregido += token

    return texto_corregido

In [18]:
nlp = spacy.load("es_core_news_sm")

# Cargamos las stopwords extra
extra_stopwords = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S6_clase_sistemas_recomendacion_II/main/data/extra_stopwords.csv', sep=',',header=None)
extra_stopwords.columns = ['stopwords']
extra_stopwords=set(extra_stopwords['stopwords'].to_list())

# Agregamos a nuestro modelo de SpaCy
nlp.Defaults.stop_words |= extra_stopwords

#print(nlp.Defaults.stop_words) # Mostrar la nueva lista de stopwords

In [19]:
# Lista de stopwords en español
stopwords_es = set(stopwords.words('spanish'))

def limpieza_texto(texto, lista_palabras=[]):
    # remover acentos
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto)
                              if unicodedata.category(c) != 'Mn')

    # remover signos de puntuación y simbolos adicionales
    texto = re.sub(r'[^\w\s]', '', texto)


    texto = texto.lower()
    # remover numeros
    texto = re.sub(r'\d+', '', texto)
    # remover saltos
    texto = re.sub(r'\n', ' ', texto)
    # remover espacios dobles
    texto = re.sub(r'\s+', ' ', texto)

    tokens = texto.split()
    tokens_filtrados = [
        t for t in tokens
        if t not in stopwords_es and t not in lista_palabras
    ]

    return ' '.join(tokens_filtrados)

In [20]:
ejemplo = blog['cuento'][3]
modificacion1 = limpieza_texto(ejemplo)
modificacion2 = corregir_texto(modificacion1)
print(modificacion2)

regreso mexico amigo comequechu conto historia dice va paseando mujer hija calles falisco entonces descubre dos pasos imponente universidad guadalajara puerta carmelita informacion turistas lee alli estan busto ganadores premio juan rulo literatura concede casa dudarlo arrastra familia pasillos vamos ver monumento cayota dice acuerdo comequechu dice cayota lugar cascar realidad nunca dirigir usando nombre apellido reales cabeza siempre cayota tambien mujer hija llaman modo toda naturalidad sono extrano segundo sustantivo frase cayota sino primero monumento malentendido embargo tenia explicacion par meses viaje norteamérica comequechu conocio noticia habia ganado certamen literario llamado juan rulo nunca supo tenia saberlo mundo dos premios mismo nombre bastante trascendente otorga francia gane aquel ano importantísimo concede mexico ganar nunca diferencia ambas distinciones abismal galardón frances premia obra puntual cuento novela corta ofrece compensación económica discreta premio m

Limpieza de datos por bloques


In [10]:
bloque1 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_1.csv')

In [11]:
bloque2 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_2.csv')

In [12]:
bloque3 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_3.csv')

In [13]:
bloque4 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_4.csv')

In [14]:
bloque5 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_5.csv')

In [21]:
bloque1['cuento_limpio'] = bloque1['cuento'].apply(lambda x: corregir_texto(limpieza_texto(x)))

# guardar datos limpios del bloque 1
bloque1.to_csv('blog_bloque_1_limpio.csv', index=False)

KeyboardInterrupt: 

In [ ]:
bloque2['cuento_limpio'] = bloque2['cuento'].apply(lambda x: corregir_texto(limpieza_texto(x)))

# guardar datos limpios del bloque 2
bloque2.to_csv('blog_bloque_2_limpio.csv', index=False)

In [ ]:
bloque3['cuento_limpio'] = bloque3['cuento'].apply(lambda x: corregir_texto(limpieza_texto(x)))

# guardar datos limpios del bloque 3
bloque3.to_csv('blog_bloque_3_limpio.csv', index=False)

In [ ]:
bloque4['cuento_limpio'] = bloque4['cuento'].apply(lambda x: corregir_texto(limpieza_texto(x)))

# guardar datos limpios del bloque 4
bloque4.to_csv('blog_bloque_4_limpio.csv', index=False)

In [ ]:
bloque5['cuento_limpio'] = bloque5['cuento'].apply(lambda x: corregir_texto(limpieza_texto(x)))

# guardar datos limpios del bloque 5
bloque5.to_csv('blog_bloque_5_limpio.csv', index=False)

In [5]:
# Cargar y unir los bloques
bloque1 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_1_limpio.csv')
bloque2 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_2_limpio.csv')
bloque3 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_3_limpio.csv')
bloque4 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_4_limpio.csv')
bloque5 = pd.read_csv('https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/datos/blog_bloque_5_limpio.csv')

df_unido = pd.concat([bloque1, bloque2, bloque3, bloque4, bloque5], ignore_index=True)


In [6]:
df_unido.head()

,titulo,fecha,cuento,cuento_limpio
0,El rincón blanco,1/11/08,De pronto yo estaba en el hogar donde pasé la ...,pronto hogar pase infancia supo primero nariz ...
1,Mínimos avances en la cama,1/24/08,"Menos la cama, todo ha mejorado en este mundo....",menos cama mejorado mundo cocinamos sopa hacie...
2,Don Marcos,2/19/08,"Dos veces, y no una, mi abuelo materno me ayud...",dos veces abuelo materno ayudo ser escritor do...
3,Los dos rulfos,3/26/08,"A su regreso de México, mi amigo Comequechu no...",regreso mexico amigo comequechu conto historia...
4,La noticia no es el perro,4/15/08,"De repente, un video de You Tube recibe un mil...",repente video you tube recibe millon visitas a...


### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones de cuentos en el blog a un usuario que leyó 'La venganza del metegol'. Para ello vamos a utilizar distintas estrategias.

#### 3.1. Recomendaciones basadas en contenidos

##### 3.1.1. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando en la distancia de coseno donde el texto este vectorizado por `CountVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# Vectorización del texto preprocesado
count_vectorizer = CountVectorizer(
    max_features=1000,        # Limitar vocabulario
    ngram_range=(1, 2),       # Unigramas y bigramas
    stop_words= None      # Remover stopwords en español
)

In [9]:
count_matrix = count_vectorizer.fit_transform(df_unido['cuento_limpio'])

In [10]:
# Cálculo de la matriz de similitud coseno
cosine_sim_count = cosine_similarity(count_matrix)

In [11]:
# Función para obtener recomendaciones ordenadas
def recomendar_cuentos_count(titulo_base, matriz_sim=cosine_sim_count, n_recomendaciones=5):
    try:
        # 1 Obtener índice del cuento base
        idx = df_unido.index[df_unido['titulo'] == titulo_base][0]

        # 2 Obtener similitudes del cuento base con todos los demás
        sim_scores = list(enumerate(matriz_sim[idx]))

        # 3 Ordenar las similitudes de mayor a menor
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4 Excluir el cuento base y seleccionar los top n recomendados
        top_indices = [i[0] for i in sim_scores[1:n_recomendaciones+1]]

        # 5 Construir DataFrame con títulos, fechas y similitud redondeada
        recomendaciones = df_unido.iloc[top_indices][['titulo', 'fecha']].copy()
        recomendaciones['Similitud'] = [round(sim_scores[i][1], 3) for i in range(1, n_recomendaciones+1)]

        # 6 Resetear índice para presentación ordenada
        recomendaciones.reset_index(drop=True, inplace=True)

        return recomendaciones

    except IndexError:
        return f"El cuento '{titulo_base}' no se encontró en el DataFrame."


In [12]:
recomendaciones = recomendar_cuentos_count('La venganza del metegol', n_recomendaciones=5)
print(recomendaciones)

                           titulo    fecha  Similitud
0   Cuento con bruja y tramontina  8/29/08      0.521
1  Una carta para Hugo Laurencena  6/21/05      0.507
2                     La revancha  6/19/09      0.497
3    La leyenda del Negro Sánchez  8/17/05      0.482
4               Electrodomésticos  4/24/15      0.481


**Explicación del procedimiento**

**Vectorización con CountVectorizer:**
Convertimos el texto limpio en vectores numéricos que representan la frecuencia de aparición de tokens (palabras y bigramas) en cada cuento. Se limita el vocabulario a las 1000 características más relevantes.

**Similitud coseno:**
Se calcula la similitud entre vectores de todos los cuentos, obteniendo valores entre 0 y 1, donde 1 indica textos idénticos y valores cercanos indican alta similitud semántica.

**Generación de recomendaciones:**
Se identifica el índice del cuento base, se ordenan los cuentos según la similitud coseno descendente, se excluye el cuento base y se seleccionan las 5 recomendaciones más similares.

**Orden y presentación:**
Las recomendaciones se presentan ordenadas de la más recomendada (mayor similitud) a la menos recomendada (quinta similitud), incluyendo el título, fecha y el valor de similitud.

##### 3.1.2. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para  el cuento 'La venganza del metegol' usando nuevamente la distancia de coseno, pero ahora vectorice el texto usando `TF-IDFVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados del punto anterior y explique sus similitudes y/o diferencias.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
# Vectorización del texto preprocesado con TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    stop_words=None
)

In [15]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df_unido['cuento_limpio'])

In [16]:
#Cálculo de la matriz de similitud coseno
cosine_sim_tfidf = cosine_similarity(tfidf_matrix)

In [17]:
# Función para obtener recomendaciones ordenadas con TF-IDF
def recomendar_cuentos_tfidf(titulo_base, matriz_sim=cosine_sim_tfidf, n_recomendaciones=5):
    try:
        # 1. Obtener índice del cuento base
        idx = df_unido.index[df_unido['titulo'] == titulo_base][0]

        # 2. Obtener similitudes del cuento base con todos los demás
        sim_scores = list(enumerate(matriz_sim[idx]))

        # 3. Ordenar las similitudes de mayor a menor
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4. Excluir el cuento base y seleccionar los top n recomendados
        top_indices = [i[0] for i in sim_scores[1:n_recomendaciones+1]]

        # 5. Construir DataFrame con títulos, fechas y similitud redondeada
        recomendaciones = df_unido.iloc[top_indices][['titulo', 'fecha']].copy()
        recomendaciones['Similitud'] = [round(sim_scores[i][1], 3) for i in range(1, n_recomendaciones+1)]

        # 6. Resetear índice para presentación ordenada
        recomendaciones.reset_index(drop=True, inplace=True)

        return recomendaciones

    except IndexError:
        return f"El cuento '{titulo_base}' no se encontró en el DataFrame."

In [18]:
recomendaciones_tfidf = recomendar_cuentos_tfidf('La venganza del metegol', n_recomendaciones=5)
print(recomendaciones_tfidf)

                                         titulo    fecha  Similitud
0                 Cuento con bruja y tramontina  8/29/08      0.464
1                Una carta para Hugo Laurencena  6/21/05      0.392
2                     La batalla del movimiento  5/28/13      0.389
3  Una línea de puntos en un libro de catecismo  7/20/05      0.387
4                    La estrategia del banderín  8/14/14      0.386


**Vectorización con TfidfVectorizer:**
Se transforma el texto preprocesado en vectores ponderados por TF-IDF, que reflejan no solo la frecuencia de las palabras en cada cuento, sino también su importancia relativa en todo el corpus. Esto penaliza palabras muy frecuentes en muchos documentos (como stopwords implícitas) y destaca términos más representativos.

**Similitud coseno:**
Se calcula la similitud entre vectores TF-IDF de todos los cuentos, obteniendo valores entre 0 y 1, donde valores más altos indican mayor similitud semántica.

**Generación de recomendaciones:**
Se identifica el índice del cuento base, se ordenan los cuentos según la similitud descendente, se excluye el cuento base y se seleccionan las 5 recomendaciones más similares.

**Presentación:**
Se devuelve un DataFrame con títulos, fechas y valores de similitud, ordenados de la recomendación más fuerte (1) a la menos fuerte (5).

A continuación tambien mencionamos las similutes y diferencias que tuvo con el anterior punto:

**Similitudes:**

- Ambas usan similitud coseno para medir cercanía entre cuentos.

- Las recomendaciones coinciden en los 2 primeros cuentos con temáticas similares.

**Diferencias:**

- Diferencia los cuentos 3 al 5 entre las dos metodologías, lo cual nos sugiere que probablemente la similitud de los cuentos 3,4 y 5 con la metodología CountVectorizer era por stop words implicitas.

- TF-IDF ofrece recomendaciones más refinadas, destacando cuentos con términos más representativos y menos ruido de palabras comunes.

- CountVectorizer puede favorecer cuentos con términos muy frecuentes, lo que a veces genera recomendaciones menos específicas.

##### 3.1.3. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando el texto vectorizado por `TF-IDFVectorizer` y la correlación como medida de similitud. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados de los puntos anteriores y explique sus similitudes y/o diferencias.

In [19]:
# Vectorización TF-IDF del texto preprocesado
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    stop_words=None
)

In [20]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df_unido['cuento_limpio']).toarray()

In [21]:
# Función para calcular matriz de similitud por correlación de Pearson
def correlacion_pearson_matriz(matriz_vectores):
    # matriz_vectores: numpy array (documentos x features)
    # Calcula la matriz de correlación entre filas (documentos)
    return np.corrcoef(matriz_vectores)

In [22]:
# 3. Calcular matriz de similitud por correlación
correlation_sim_matrix = correlacion_pearson_matriz(tfidf_matrix)

In [23]:
# Función para obtener recomendaciones ordenadas por correlación
# -----------------------------------------------------------------
def recomendar_cuentos_tfidf_correlacion(titulo_base, matriz_sim=correlation_sim_matrix, n_recomendaciones=5):
    try:
        # 1. Obtener índice del cuento base
        idx = df_unido.index[df_unido['titulo'] == titulo_base][0]

        # 2. Obtener similitudes (correlaciones) del cuento base con todos los demás
        sim_scores = list(enumerate(matriz_sim[idx]))

        # 3. Ordenar similitudes de mayor a menor
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 4. Excluir el cuento base y seleccionar los top n recomendados
        top_indices = [i[0] for i in sim_scores[1:n_recomendaciones+1]]

        # 5. Construir DataFrame con títulos, fechas y similitud redondeada
        recomendaciones = df_unido.iloc[top_indices][['titulo', 'fecha']].copy()
        recomendaciones['Similitud'] = [round(sim_scores[i][1], 3) for i in range(1, n_recomendaciones+1)]

        # 6. Resetear índice para presentación ordenada
        recomendaciones.reset_index(drop=True, inplace=True)

        return recomendaciones

    except IndexError:
        return f"El cuento '{titulo_base}' no se encontró en el DataFrame."


In [24]:
recomendaciones_correlacion = recomendar_cuentos_tfidf_correlacion('La venganza del metegol', n_recomendaciones=5)
print(recomendaciones_correlacion)

                                         titulo    fecha  Similitud
0                 Cuento con bruja y tramontina  8/29/08      0.375
1  Una línea de puntos en un libro de catecismo  7/20/05      0.314
2                    La estrategia del banderín  8/14/14      0.308
3                     La batalla del movimiento  5/28/13      0.297
4                Una carta para Hugo Laurencena  6/21/05      0.292


**Procedimiento realizado**

Se utilizó el vectorizador TF-IDFVectorizer para transformar los textos de los cuentos en vectores numéricos, donde cada dimensión representa la importancia relativa de una palabra en un documento respecto al resto del corpus. Luego, se empleó la correlación como métrica de similitud para medir qué tan similares son los cuentos entre sí con base en su contenido textual.

Para obtener las recomendaciones:

1. Se vectorizó todo el corpus.

2. Se extrajo el vector correspondiente al cuento 'La venganza del metegol'.

3. Se calcularon las correlaciones de este vector con los vectores de los demás cuentos.

4. Se ordenaron de mayor a menor correlación, omitiendo el propio cuento.

5. Se seleccionaron los 5 cuentos con mayor similitud para generar el ranking.


**Similitudes:**

- Algunos cuentos recomendados por TF-IDF también aparecieron como cercanos temáticamente en el modelo LDA, lo que sugiere que hay coherencia entre el contenido literal (palabras usadas) y los temas latentes.

- Ambos métodos posicionaron bien cuentos con un enfoque narrativo deportivo o de conflicto.

**Diferencias:**

- TF-IDF se basa en la coincidencia de palabras específicas, por lo que puede priorizar cuentos que comparten el vocabulario exacto, aunque hablen de temas distintos.

- En cambio, LDA capta relaciones temáticas más abstractas aunque se usen diferentes palabras, lo cual puede generar recomendaciones más conceptuales o generales.

- Por tanto, algunas recomendaciones por LDA no coinciden con las de TF-IDF, ya que su enfoque es más semántico que léxico.

##### 3.2. Recomendaciones basadas en temas

Usando modelado de temas con LDA, encuentre los temas subyacentes en el blog. Explique como eligió el numero óptimo de temas. Utilizando el tema asignado al cuento 'La venganza del metegol' y la probabilidad de pertenecer a este tema genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para este cuento. Explique el procedimiento que realizó. Compare con los resultados encontrados anteriormente y explique sus similitudes y/o diferencias. (Esto puede tomar mucho tiempo y requerir mucha capacidad computacional, puede aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [1]:
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Preprocesamiento para LDA: tokenización y limpieza
def preprocess_lda(text):
    tokens = [word for word in gensim.utils.simple_preprocess(text, deacc=True)
              if word not in stop_words and len(word) > 3]
    return tokens

texts_lda = df_unido['cuento_limpio'].map(preprocess_lda).tolist()


In [28]:
# Crear diccionario y corpus para LDA
dictionary = corpora.Dictionary(texts_lda)
corpus = [dictionary.doc2bow(text) for text in texts_lda]

In [29]:
# Función para entrenar modelos LDA y calcular coherencia
def evaluar_lda(num_topics):
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=100,
        update_every=1,
        chunksize=100,
        passes=10,
        alpha='auto',
        per_word_topics=True
    )
    coherencemodel = CoherenceModel(model=lda_model, texts=texts_lda, dictionary=dictionary, coherence='c_v')
    return lda_model, coherencemodel.get_coherence()


In [30]:
# Evaluar modelos para varios números de temas
resultados = []
for k in range(2, 11):
    modelo, coherencia = evaluar_lda(k)
    resultados.append((k, coherencia))
    print(f'Número de temas: {k}, Coherencia: {coherencia:.4f}')


Número de temas: 2, Coherencia: 0.2588
Número de temas: 3, Coherencia: 0.3001
Número de temas: 4, Coherencia: 0.3550
Número de temas: 5, Coherencia: 0.3222
Número de temas: 6, Coherencia: 0.3594
Número de temas: 7, Coherencia: 0.3794
Número de temas: 8, Coherencia: 0.4288
Número de temas: 9, Coherencia: 0.4403
Número de temas: 10, Coherencia: 0.3361


In [31]:
# Seleccionar el número óptimo de temas (el que maximiza coherencia)
num_temas_optimo = max(resultados, key=lambda x: x[1])[0]
print(f'Número óptimo de temas seleccionado: {num_temas_optimo}')


Número óptimo de temas seleccionado: 9


In [32]:
# Entrenar modelo final con número óptimo de temas
lda_model_final = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_temas_optimo,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=15,
    alpha='auto',
    per_word_topics=True
)

In [33]:
# Obtener índice del cuento base
idx_cuento = df_unido.index[df_unido['titulo'] == 'La venganza del metegol'][0]

In [34]:
# Obtener distribución de temas para el cuento base
temas_cuento = lda_model_final.get_document_topics(corpus[idx_cuento], minimum_probability=0)

In [35]:
# Identificar tema principal (el de mayor probabilidad)
tema_principal = max(temas_cuento, key=lambda x: x[1])[0]
print(f'Tema principal asignado a "La venganza del metegol": {tema_principal}')


Tema principal asignado a "La venganza del metegol": 6


In [36]:
# Obtener la probabilidad de pertenencia de todos los cuentos al tema principal
prob_tema_principal = []
for i, doc_bow in enumerate(corpus):
    doc_topics = lda_model_final.get_document_topics(doc_bow, minimum_probability=0)
    # Extraer probabilidad del tema principal
    prob = dict(doc_topics).get(tema_principal, 0)
    prob_tema_principal.append((i, prob))


In [37]:
# Ordenar cuentos por probabilidad descendente, excluyendo el cuento base
prob_tema_principal = sorted(prob_tema_principal, key=lambda x: x[1], reverse=True)
prob_tema_principal = [x for x in prob_tema_principal if x[0] != idx_cuento]

In [38]:
# Seleccionar top 5 recomendaciones
top5_indices = [x[0] for x in prob_tema_principal[:5]]
top5_probs = [x[1] for x in prob_tema_principal[:5]]


In [39]:
# Construir DataFrame con resultados
recomendaciones_lda = df_unido.iloc[top5_indices][['titulo', 'fecha']].copy()
recomendaciones_lda['Probabilidad_Tema'] = [round(p, 4) for p in top5_probs]
recomendaciones_lda.reset_index(drop=True, inplace=True)

print(recomendaciones_lda)

                          titulo     fecha  Probabilidad_Tema
0        ¿Me agregás como amiga?    5/8/15             0.9982
1      La tarde que fuimos ricos   1/19/04             0.9976
2  Un hombre alto bajo la lluvia   2/20/04             0.9974
3    Una docena de tortas negras   5/19/05             0.9971
4                   Uno que pide  12/23/03             0.9969


**Explicación del procedimiento**

Se preprocesaron los textos para LDA mediante tokenización, eliminación de stopwords y palabras cortas.

Se creó un diccionario y corpus en formato BoW para la librería gensim.

Se entrenaron modelos LDA con diferentes números de temas (2 a 10) y se evaluó la coherencia para elegir el número óptimo.

Se entrenó el modelo final con el número óptimo de temas.

Para el cuento "La venganza del metegol", se obtuvo la distribución de temas y se identificó su tema principal.

Se calcularon las probabilidades de pertenencia de todos los cuentos a ese tema principal.

Se seleccionaron los 5 cuentos con mayor probabilidad en ese tema, excluyendo el cuento base, como recomendaciones.

### 4. Recomendaciones generales

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para la entrada elegida? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

**¿Qué procedimiento generó las mejores recomendaciones?**

En términos de precisión semántica, el modelo LDA parece ofrecer las recomendaciones más significativas temáticamente, ya que agrupa cuentos por tópicos más profundos, incluso cuando el vocabulario varía. Esto permite una mejor generalización y evita sesgos por palabras frecuentes que puedan no ser centrales al contenido.

No obstante, el método basado en TF-IDF y correlación también es útil, especialmente cuando se busca similitud literal y superficial, como tono, estilo o terminología.

Por tanto, la mejor opción dependera al final del objetivo de similitud:

- Si quisiera mantener un estilo y contexto inmediato → TF-IDF.

- Si quisiera explorar cuentos temáticamente similares → LDA.

**¿Cómo implementaría una evaluación objetiva de estas recomendaciones?**

1. Validación manual por expertos: Pedir a un grupo de lectores (idealmente conocedores de literatura infantil o juvenil) que califiquen la pertinencia de las recomendaciones en relación con el cuento base.

2. Métricas de evaluación tipo IR (Information Retrieval):

- Precisión: Proporción de cuentos relevantes entre los k recomendados.

- NDCG (Normalized Discounted Cumulative Gain): Toma en cuenta la posición del cuento relevante en la lista.

- Diversidad temática: Para evitar recomendaciones muy similares y promover variedad.

3. Encuestas a usuarios finales: Implementar un sistema donde los usuarios valoren las recomendaciones recibidas, y así recopilar datos empíricos de su utilidad.

4. Matriz de confusión o curva ROC (si se cuenta con etiquetas de similitud previa) para comparar el rendimiento de ambos métodos frente a un “gold standard”.